# Credentials with IAM Role

### and Rasterio and GDAL can be difficult

[Wednesday 6:39 AM] Signell, Richard P
# CHS Sam Congdon is a Rock Star
(1 liked)​[Wednesday 6:40 AM] Signell, Richard P
    
        --role-arn arn:aws:iam::376067480372:role/eks-pangeo-node \
        --role-session-name gdal-session-$(date +%s) \
        --web-identity-token file:///var/run/secrets/eks.amazonaws.com/serviceaccount/token                                                                                          




So if you run do source set_gdal.sh with this script (that I made based on a script from Matt Hanson):

```
#!/bin/bash
# get temporary keys for GDAL based on role using AWS CLI
AWS_STS_RESPONSE=`aws sts assume-role-with-web-identity \
        --role-arn arn:aws:iam::376067480372:role/eks-pangeo-node \
        --role-session-name gdal-session-$(date +%s) \
        --web-identity-token file:///var/run/secrets/eks.amazonaws.com/serviceaccount/token`
# parse CLI response using jq and grep
export AWS_ACCESS_KEY_ID=`echo ${AWS_STS_RESPONSE} | jq -r '.Credentials.AccessKeyId'`
export AWS_SECRET_ACCESS_KEY=`echo ${AWS_STS_RESPONSE} | jq -r '.Credentials.SecretAccessKey'`
export AWS_SESSION_TOKEN=`echo ${AWS_STS_RESPONSE} | jq -r '.Credentials.SessionToken'`
export AWS_REGION=`echo ${AWS_STS_RESPONSE} | jq -r '.Provider' | grep -oP '(?<=eks.).*?(?=.amazo
naws)'`
# for GDAL<3.2
export AWS_DEFAULT_REGION=$AWS_REGION
```
 

In [1]:
! env | grep AWS

AWS_ROLE_ARN=arn:aws:iam::376067480372:role/eks-pangeo-node
AWS_WEB_IDENTITY_TOKEN_FILE=/var/run/secrets/eks.amazonaws.com/serviceaccount/token


In [3]:
! aws sts assume-role-with-web-identity --role-arn ${AWS_ROLE_ARN} \
    --web-identity-token ${AWS_WEB_IDENTITY_TOKEN_FILE} \
    --role-session-name gdal-session-$(date +%s)
    


An error occurred (InvalidIdentityToken) when calling the AssumeRoleWithWebIdentity operation: The ID Token provided is not a valid JWT. (You may see this error if you sent an Access Token)


In [9]:
%%bash
AWS_STS_RESPONSE=`aws sts assume-role-with-web-identity \
        --role-arn arn:aws:iam::376067480372:role/eks-pangeo-node \
        --role-session-name gdal-session-$(date +%s) \
        --web-identity-token file:///var/run/secrets/eks.amazonaws.com/serviceaccount/token`

echo ${AWS_STS_RESPONSE} | python3 -m json.tool> my_cred.json

In [19]:
import json
file_j = 'my_cred.json'
with open(file_j) as json_file: 
    data = json.load(json_file) 
    
key = data['Credentials']['AccessKeyId']

secret = data['Credentials']['SecretAccessKey']

session_token = data['Credentials']['SessionToken']



In [17]:
import os

env_var = os.environ 

env_var



environ{'CONDA_SHLVL': '1',
        'LC_ALL': 'C.UTF-8',
        'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-butzer%40contractor.usgs.gov',
        'CONDA_EXE': '/srv/conda/bin/conda',
        'PROXY_API_PORT': 'tcp://172.20.190.162:8001',
        'GDAL_DATA': '/srv/conda/envs/pangeo/share/gdal',
        'PROXY_API_PORT_8001_TCP_PORT': '8001',
        'JUPYTERHUB_ADMIN_ACCESS': '1',
        'LANG': 'C.UTF-8',
        'HUB_PORT_8081_TCP_PROTO': 'tcp',
        'HUB_SERVICE_HOST': '172.20.135.17',
        'HOSTNAME': 'jupyter-butzer-40contractor-2eusgs-2egov',
        'PANGEO_ENV': 'pangeo-notebook',
        'NB_UID': '1000',
        'GSETTINGS_SCHEMA_DIR_CONDA_BACKUP': '',
        'CPL_ZIP_ENCODING': 'UTF-8',
        'PROXY_PUBLIC_PORT_80_TCP_ADDR': '172.20.190.184',
        'PROXY_PUBLIC_PORT_443_TCP_ADDR': '172.20.190.184',
        'CONDA_PREFIX': '/srv/conda/envs/pangeo',
        'PROXY_PUBLIC_PORT_80_TCP_PORT': '80',
        'CONDA_DIR': '/srv/conda',
        'CONDA_VERSION': '4.7.12.1'

In [20]:

os.environ['AWS_ACCESS_KEY_ID'] = key
os.environ['AWS_SECRET_ACCESS_KEY'] = secret
os.environ['AWS_SESSION_TOKEN'] = session_token

os.environ['AWS_DEFAULT_REGION'] = 'us-west-2'

os.environ['AWS_REQUEST_PAYER'] = 'requester'


In [21]:
! rio --help

Usage: rio [OPTIONS] COMMAND [ARGS]...

  Rasterio command line interface.

Options:
  -v, --verbose           Increase verbosity.
  -q, --quiet             Decrease verbosity.
  --aws-profile TEXT      Select a profile from the AWS credentials file
  --aws-no-sign-requests  Make requests anonymously
  --aws-requester-pays    Requester pays data transfer costs
  --version               Show the version and exit.
  --gdal-version
  --help                  Show this message and exit.

Commands:
  blocks     Write dataset blocks as GeoJSON features.
  bounds     Write bounding boxes to stdout as GeoJSON.
  calc       Raster data calculator.
  clip       Clip a raster to given bounds.
  cogeo      † Warning: could not load plugin. See `rio cogeo --help`.
  convert    Copy and convert raster dataset.
  edit-info  Edit dataset metadata.
  env        Print information about the Rasterio environment.
  gcps       Print ground control points as GeoJSON.
  info       Print information about a da

## --request-payer=requester

You must authenticate all requests involving Requester Pays buckets. The request authentication enables Amazon S3 to identify and charge the requester for their use of the Requester Pays bucket. 

[5/6 2:41 PM] Congdon, Samuel F
    yeah, you have to add the --request-payer=requester argument for CLI commands


In [22]:
%env

{'CONDA_SHLVL': '1',
 'LC_ALL': 'C.UTF-8',
 'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-butzer%40contractor.usgs.gov',
 'CONDA_EXE': '/srv/conda/bin/conda',
 'PROXY_API_PORT': 'tcp://172.20.190.162:8001',
 'GDAL_DATA': '/srv/conda/envs/pangeo/share/gdal',
 'PROXY_API_PORT_8001_TCP_PORT': '8001',
 'JUPYTERHUB_ADMIN_ACCESS': '1',
 'LANG': 'C.UTF-8',
 'HUB_PORT_8081_TCP_PROTO': 'tcp',
 'HUB_SERVICE_HOST': '172.20.135.17',
 'HOSTNAME': 'jupyter-butzer-40contractor-2eusgs-2egov',
 'PANGEO_ENV': 'pangeo-notebook',
 'NB_UID': '1000',
 'GSETTINGS_SCHEMA_DIR_CONDA_BACKUP': '',
 'CPL_ZIP_ENCODING': 'UTF-8',
 'PROXY_PUBLIC_PORT_80_TCP_ADDR': '172.20.190.184',
 'PROXY_PUBLIC_PORT_443_TCP_ADDR': '172.20.190.184',
 'CONDA_PREFIX': '/srv/conda/envs/pangeo',
 'PROXY_PUBLIC_PORT_80_TCP_PORT': '80',
 'CONDA_DIR': '/srv/conda',
 'CONDA_VERSION': '4.7.12.1',
 'PROXY_PUBLIC_SERVICE_PORT': '80',
 '_CE_M': '',
 'KUBERNETES_PORT_443_TCP_PROTO': 'tcp',
 'JUPYTERHUB_ACTIVITY_URL': 'http://172.20.135.17:8081/hub/ap

In [23]:
! aws s3 ls dev-et-data/


An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied


In [24]:
%%bash

export awr='aws --request-payer=requester'

${awr} s3 ls dev-et-data/

                           PRE NDVI_filled/
                           PRE compressed/
                           PRE level-1/
                           PRE t50n-90e/
                           PRE v1DRB_outputs/
2020-05-05 23:59:59         62 Readme-bucket.md
2020-05-05 23:50:00         13 tony.txt


In [27]:
! aws s3 ls --request-payer=requester dev-et-data/compressed/NDVI_filled/2001/

2020-04-29 23:36:13 1202824254 2001001.250_m_NDVI.tif
2020-04-29 23:36:41       2786 2001001.250_m_NDVI.tif.aux.xml
2020-04-29 23:36:46 1216430032 2001002.250_m_NDVI.tif
2020-04-29 23:36:58       2785 2001002.250_m_NDVI.tif.aux.xml
2020-04-29 23:36:34 1214412397 2001003.250_m_NDVI.tif
2020-04-29 23:36:44       2780 2001003.250_m_NDVI.tif.aux.xml
2020-04-29 23:35:06 1215553227 2001004.250_m_NDVI.tif
2020-04-29 23:35:27       2783 2001004.250_m_NDVI.tif.aux.xml
2020-04-29 23:36:45 1211083406 2001005.250_m_NDVI.tif
2020-04-29 23:36:57       2785 2001005.250_m_NDVI.tif.aux.xml
2020-04-29 23:35:54 1215583226 2001006.250_m_NDVI.tif
2020-04-29 23:36:02       2784 2001006.250_m_NDVI.tif.aux.xml
2020-04-29 23:36:29 1214085039 2001007.250_m_NDVI.tif
2020-04-29 23:36:39       2779 2001007.250_m_NDVI.tif.aux.xml
2020-04-29 23:36:39 1215683446 2001008.250_m_NDVI.tif
2020-04-29 23:36:52       2775 2001008.250_m_NDVI.tif.aux.xml
2020-04-29 23:35:30 1206977319 2001009.250_m_NDVI.tif
2020-04-29 23:35:4

In [28]:
! gdalinfo /vsis3/dev-et-data/compressed/NDVI_filled/2001/2001366.250_m_NDVI.tif

Driver: GTiff/GeoTIFF
Files: /vsis3/dev-et-data/compressed/NDVI_filled/2001/2001366.250_m_NDVI.tif
       /vsis3/dev-et-data/compressed/NDVI_filled/2001/2001366.250_m_NDVI.tif.aux.xml
Size is 49667, 14416
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-155.572382657999981,49.999999996000085)
Pixel Size = (0.002081004500000,-0.002081004500000)
Metadata:
  AREA_OR_POINT=Area
  DataType=Generic
  OVR_RESAMPLING_ALG=NEAREST
Image Structure Metadata:
  COMPRESSION=DEFLATE
  

In [29]:
! rio info /vsis3/dev-et-data/compressed/NDVI_filled/2001/2001366.250_m_NDVI.tif

{"blockxsize": 512, "blockysize": 512, "bounds": [-155.57238265799998, 20.000239124000082, -52.215132156499976, 49.999999996000085], "colorinterp": ["gray"], "compress": "deflate", "count": 1, "crs": "EPSG:4326", "descriptions": [null], "driver": "GTiff", "dtype": "float32", "height": 14416, "indexes": [1], "interleave": "band", "lnglat": [-103.89375740724998, 35.00011956000009], "mask_flags": [["nodata"]], "nodata": -3.4028234663852886e+38, "res": [0.0020810045, 0.0020810045], "shape": [14416, 49667], "tiled": true, "transform": [0.0020810045, 0.0, -155.57238265799998, 0.0, -0.0020810045, 49.999999996000085, 0.0, 0.0, 1.0], "units": [null], "width": 49667}
